# Text to XML

## Import

### Libraries

In [5]:
import os
import pandas as pd
import codecs
from lxml import etree

from backend import *

### Definitions

In [10]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Primary Texts

In [11]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r') as f: 
            textytext = f.read()
        subDf = pd.DataFrame({'author': info.group(1),'pub_year': int(info.group(2)), 'title': info.group(3), 'text': textytext}, index=[1])
        libDf = pd.concat([libDf, subDf], ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).reset_index().drop(['index'], axis=1)
libDf.index.name = 'w_id'
libTextsDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,pub_year,title
w_id,,,
0,gorkii,1900,troe
1,andreev,1903,zhizn-vasiliia-fiveiskogo
2,andreev,1904,gubernator
3,andreev,1905,k-zvezdam
4,andreev,1905,khristiane
5,andreev,1905,tak-bylo
6,gorkii,1906,mat
7,andreev,1906,savva-ignis-sanat
8,andreev,1907,iuda-iskariot


In [12]:
motherTextDf = textRegularize(libTextsDf, 6)
motherTextDf

,chapID,para,text,paraID
0,1,1,"Каждый день над рабочей слободкой, в ды...",1
1,1,2,"Вечером, когда садилось солнце, и на ст...",2
2,1,3,"День проглочен фабрикой, машины высосал...",3
3,1,4,"По праздникам спали часов до десяти, по...",4
4,1,5,"Усталость, накопленная годами, лишала л...",5
...,...,...,...,...
4490,58,90,Ударили по руке.,4432
4491,58,91,"-- Только злобы накопите, безумные! На ...",4433
4492,58,92,Жандарм схватил ее за горло и стал души...,4434
4493,58,93,-- Несчастные...,4435


In [13]:
confessionTextDf = textRegularize(libTextsDf, 10)
confessionTextDf

,text,paraID
0,...Позвольте рассказать жизнь мою; врем...,1
1,"Я -- крапивник, подкидыш, незаконный че...",2
2,"У Данилы прожил я до четырёх лет, но он...",3
3,Четырёх лет взял меня к себе дьячок Лар...,4
4,"От людей в стороне стоял, жил бедно, на...",5
...,...,...
2357,Окрыляет он жизнь её величием деяний и ...,2356
2358,"-- Ты еси мой бог и творец всех богов, ...",2357
2359,-- Да не будут миру бози инии разве теб...,2358
2360,-- Тако верую и исповедую!,2359


### Make XML

In [ ]:
root = etree.Element("text")
print(root.tag)
pt = ch = cn = pa = pn = 0
nameDict = motherDf.chap.to_dict()
for part in motherXmlDf.part.unique():
    print(f"Part {part}")
    root.append(etree.Element("part", n=str(part), name=str(part)))
    chapList = motherXmlDf.loc[motherXmlDf['part'] == part].chapID.unique()
    #print(chapList)
    for chap in chapList:
        #print(f"Chap {chap}")
        root[pt].append(etree.Element("chapter", n=str(cn+1), name=nameDict.get(chap)))
        paraList = motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chapID'] == chap)].index
        #print(paraList)
        for paragraph in paraList:
            #print(f"Paragraph {paragraph}")
            root[pt][ch].append(etree.Element("paragraph", n=str(pn+1), name=str(pa+1)))
            paraText = motherXmlDf.loc[paragraph].text
            #print(f"paraText: {paraText}")
            #print(f"pt = {pt}; ch = {ch}; paragraph = {paragraph}")
            root[pt][ch][pa].text = paraText
            pa+=1
            pn+=1
        pa=0
        ch+=1
        cn+=1
    ch=0
    pt+=1
#t+=1
#print(etree.tostring(root, pretty_print=True, xml_declaration=True))
#etree.ElementTree(root).write('../site/texts/mother.xml', pretty_print=True, xml_declaration=True, encoding='windows-1251')

In [15]:
def make_textXmlDf(textTextDf):
    textXmlDf = textTextDf['text'].str.split(' \n', expand=True).stack().to_frame().reset_index().rename(columns={'level_0':'chapID','level_1':'para',0:'text'})
    textXmlDf = textXmlDf.loc[~textXmlDf.text.str.contains(r"^\W*$", regex=True)]
    textXmlDf['text'] = textXmlDf.text.str.replace('\n|\s{2,}', '')
    textXmlDf['part'] = textXmlDf.chapID.apply(lambda x: int('1') if x < 30 else int('2'))
    textXmlDf['chap'] = textXmlDf.chapID.map(motherTextDf['chapID'].to_dict())
    textXmlDf['para'] = textXmlDf['para'].apply(lambda x: x+1)
    textXmlDf['paraID'] = range(1, len(textXmlDf)+1)
    textXmlDf = motherXmlDf.reset_index().drop('index', axis=1)
    
    return textXmlDf

In [17]:
motherXmlDf = motherTextDf['text'].str.split(' \n', expand=True).stack().to_frame().reset_index().rename(columns={'level_0':'chapID','level_1':'para',0:'text'})
motherXmlDf = motherXmlDf.loc[~motherXmlDf.text.str.contains(r"^\W*$", regex=True)]
motherXmlDf['text'] = motherXmlDf.text.str.replace('\n|\s{2,}', '')
motherXmlDf['part'] = motherXmlDf.chapID.apply(lambda x: int('1') if x < 30 else int('2'))
motherXmlDf['chap'] = motherXmlDf.chapID.map(motherTextDf['chapID'].to_dict())
motherXmlDf['para'] = motherXmlDf['para'].apply(lambda x: x+1)
motherXmlDf['paraID'] = range(1, len(motherXmlDf)+1)
motherXmlDf = motherXmlDf.reset_index().drop('index', axis=1)

motherXmlDf

,chapID,para,text,part,chap,paraID
0,0,1,"Каждый день над рабочей слободкой, в ды...",1,1,1
1,1,1,"Вечером, когда садилось солнце, и на ст...",1,1,2
2,2,1,"День проглочен фабрикой, машины высосал...",1,1,3
3,3,1,"По праздникам спали часов до десяти, по...",1,1,4
4,4,1,"Усталость, накопленная годами, лишала л...",1,1,5
...,...,...,...,...,...,...
4431,4490,1,Ударили по руке.,2,58,4432
4432,4491,1,"-- Только злобы накопите, безумные! На ...",2,58,4433
4433,4492,1,Жандарм схватил ее за горло и стал души...,2,58,4434
4434,4493,1,-- Несчастные...,2,58,4435


In [16]:
make_textXmlDf(confessionTextDf)

,chapID,para,text,part,chap,paraID
0,0,1,"Каждый день над рабочей слободкой, в ды...",1,1,1
1,1,1,"Вечером, когда садилось солнце, и на ст...",1,1,2
2,2,1,"День проглочен фабрикой, машины высосал...",1,1,3
3,3,1,"По праздникам спали часов до десяти, по...",1,1,4
4,4,1,"Усталость, накопленная годами, лишала л...",1,1,5
...,...,...,...,...,...,...
4431,4490,1,Ударили по руке.,2,58,4432
4432,4491,1,"-- Только злобы накопите, безумные! На ...",2,58,4433
4433,4492,1,Жандарм схватил ее за горло и стал души...,2,58,4434
4434,4493,1,-- Несчастные...,2,58,4435


In [47]:
root = etree.Element("prose")
print(root.tag)
pt = ch = pa = 1
#TestList = [x for x in bibliiaDf.test.unique()]
#for test in range(len(TestList)): 
#    root.append(etree.Element("t", n=str(t), name=TestList[test]))

for part in motherXmlDf.part.unique(): 
    print(f"Part {part}")
    ChapList = list(motherXmlDf.loc[motherXmlDf['part'] == str(part)].index)
    for chap in ChapList:
        print(f"Chap {chap}")
        root[int(part)-1].append(etree.Element("chap", n=str(ch), name=str(chap)))
        ParaList = list(motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chap'] == chap)].index)
        for para in ParaList:
            print(f"Para {para}")
            root[part][chap].append(etree.Element("para", n=str(pa), name=str(para)))
            para_text = motherXmlDf.loc[pa].text
            print(f"Paratext: {para_text}")
            

prose
Part 1
Part 2
